In [51]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import gensim
import spacy


# Загрузка библиотеки английского языка
nlp = spacy.load('en_core_web_sm')

Загрузка данных:

In [52]:
train_df = pd.read_csv('products_sentiment_train.tsv',sep='\t',header=None)
test_df = pd.read_csv('products_sentiment_test.tsv',sep='\t')

In [53]:
train_df.head()

0  1
0          2 . take around 10,000 640x480 pictures .  1
1  i downloaded a trial version of computer assoc...  1
2  the wrt54g plus the hga7t is a perfect solutio...  1
3  i dont especially like how music files are uns...  0
4  i was using the cheapie pail ... and it worked...  1

Создание процедуры для обработки текста. Для начала с помощью модуля gensim удалим лишние пробелы, пунктуацию и цифры. Затем с помощью модуля spacy выполним лемматизацию слов.

In [54]:
def preprocess_str(text): 
    filters = [gensim.parsing.preprocessing.strip_multiple_whitespaces,
              gensim.parsing.preprocessing.strip_punctuation,
              gensim.parsing.preprocessing.strip_numeric]
    
    text_for_lemma = ' '.join(gensim.parsing.preprocessing.preprocess_string(text,filters=filters))
    return ' '.join([token.text if token.lemma_ == '-PRON-' else token.lemma_ for token in nlp(text_for_lemma)])
    

train_df['prep_text'] = train_df[0].apply(preprocess_str)
test_df['prep_text'] = test_df['text'].apply(preprocess_str)

Создадим простой пайплайн из TfidfVectorizer и LogisticRegression, проверим его на кросс_валидации:

In [58]:
vect = TfidfVectorizer(ngram_range=(2,8),analyzer ='char',max_df=0.8,min_df=0.001)
clf = LogisticRegression(C=2)

simple_pipeline = Pipeline([('vect', vect), ('clf', clf)])

cross_val_score(simple_pipeline,train_df['prep_text'],train_df[1],scoring='roc_auc').mean()

0.8652094168401806

Обучим модель на всех данных и выгрузим предсказания для тестового набора:

In [59]:
simple_pipeline.fit(train_df['prep_text'],train_df[1])

test_df['y'] = simple_pipeline.predict(test_df['prep_text'])
test_df[['Id','y']].to_csv('res.csv',index=False)

Сохраним обученную модель для следующих заданий:

In [57]:
from sklearn.externals import joblib

joblib.dump(simple_pipeline, 'model.pkl') 

['model.pkl']